<a href="https://colab.research.google.com/github/rafabandoni/nfl-predict/blob/main/notebooks/02_nfl_predict_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

# 02. Feature Engineering

## Reading data

In [3]:
PATH = 'https://raw.githubusercontent.com/rafabandoni/nfl-predict/refs/heads/main/data/output/'

In [4]:
treated_df = pd.read_parquet(PATH + 'treated_df.parquet')
treated_df.head()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_scrm_plys_home_,defense_fumbles_ff_home_,defense_fumbles_fr_home_,defense_interceptions_int_yds_home_,defense_interceptions_lng_home_,defense_passing_cmp__perc_home_,defense_passing_rate_home_,defense_receiving_yds/rec_home_,defense_receiving_pdef_home_,defense_rushing_ypc_home_,defense_rushing_td_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_scoring_sfty_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_scrm_plys_home_,offense_receiving_yds/rec_home_,offense_receiving_lng_home_,offense_receiving_rec_fum_home_,offense_rushing_ypc_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_fum_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_lng_home_,special_teams_kickoff_returns_fc_home_,special_teams_kickoff_returns_fum_home_,special_teams_kickoff_returns_fg_blk_home_,special_teams_kickoff_returns_xp_blk_home_,special_teams_kickoffs_ret_home_,special_teams_kickoffs_ret_avg_home_,special_teams_kickoffs_osk_home_,special_teams_kickoffs_osk_rec_home_,special_teams_kickoffs_oob_home_,special_teams_kickoffs_td_home_,special_teams_punt_returns_avg_home_,special_teams_punt_returns_ret_home_,special_teams_punt_returns_pret_t_home_,special_teams_punt_returns_40+_home_,special_teams_punt_returns_lng_home_,special_teams_punt_returns_fc_home_,special_teams_punt_returns_fum_home_,special_teams_punt_returns_p_blk_home_,defense_downs_3rd_att_away_,defense_downs_3rd_md_away_,defense_downs_scrm_plys_away_,defense_fumbles_ff_away_,defense_fumbles_fr_away_,defense_interceptions_int_yds_away_,defense_interceptions_lng_away_,defense_passing_cmp__perc_away_,defense_passing_rate_away_,defense_receiving_yds/rec_away_,defense_receiving_pdef_away_,defense_rushing_ypc_away_,defense_rushing_td_away_,defense_rushing_20+_away_,defense_rushing_40+_away_,defense_rushing_lng_away_,defense_scoring_sfty_away_,defense_tackles_solo_away_,offense_downs_3rd_att_away_,offense_downs_3rd_md_away_,offense_downs_scrm_plys_away_,offense_receiving_yds/rec_away_,offense_receiving_lng_away_,offense_receiving_rec_fum_away_,offense_rushing_ypc_away_,offense_rushing_20+_away_,offense_rushing_40+_away_,offense_rushing_lng_away_,offense_rushing_rush_fum_away_,offense_scoring_2-pt_away_,special_teams_field_goals_fg__perc_away_,special_teams_field_goals_1-19_>_percent_a_m_away_,special_teams_field_goals_20-29_>_percent_a_m_away_,special_teams_field_goals_30-39_>_percent_a_m_away_,special_teams_field_goals_40-49_>_percent_a_m_away_,special_teams_field_goals_50-59_>_percent_a_m_away_,special_teams_field_goals_60+_>_percent_a_m_away_,special_teams_field_goals_lng_away_,special_teams_field_goals_fg_blk_away_,special_teams_kickoff_returns_avg_away_,special_teams_kickoff_returns_kret_td_away_,special_teams_kickoff_returns_40+_away_,special_teams_kickoff_returns_lng_away_,special_teams_kickoff_returns_fc_away_,special_teams_kickoff_returns_fum_away_,special_teams_kickoff_returns_fg_blk_away_,special_teams_kickoff_returns_xp_blk_away_,special_teams_kickoffs_ret_away_,special_teams_kickoffs_ret_avg_away_,special_teams_kickoffs_osk_away_,special_teams_kickoffs_osk_rec_away_,special_teams_kickoffs_oob_away_,specia

In [5]:
featured_df = treated_df.copy()

## Lag

* Lag is a feature with time information

### Last score lag playing home and away

In [6]:
def get_lag_by_team_date(data, lag, home=True):
  final_lagged_df = pd.DataFrame()
  is_home_team = 'team_home' if home==True else 'team_away'
  is_home_score = 'score_home' if home==True else 'score_away'
  new_column_name = 'home_lag_' if home==True else 'away_lag_'

  for team in data[is_home_team].unique():
    lag_df = data[data[is_home_team] == team]
    lag_df = lag_df[['schedule_date', is_home_team, is_home_score]]
    lag_df = lag_df.sort_values([is_home_team, 'schedule_date'])
    lag_df = lag_df.set_index([is_home_team, 'schedule_date']).shift(lag).reset_index()
    final_lagged_df = pd.concat([final_lagged_df, lag_df], axis=0, ignore_index=True)
  final_lagged_df.rename(columns={
      is_home_score : f'{new_column_name}{lag}'
  }, inplace=True)
  return final_lagged_df

In [7]:
df_home_lag_1 = get_lag_by_team_date(treated_df, 1)
df_home_lag_2 = get_lag_by_team_date(treated_df, 2)
df_home_lag_3 = get_lag_by_team_date(treated_df, 3)

df_home_lag = df_home_lag_1.merge(df_home_lag_2,
                                  on=['team_home', 'schedule_date'],
                                  how='left')
df_home_lag = df_home_lag.merge(df_home_lag_3,
                                  on=['team_home', 'schedule_date'],
                                  how='left')

In [8]:
df_away_lag_1 = get_lag_by_team_date(treated_df, 1, home=False)
df_away_lag_2 = get_lag_by_team_date(treated_df, 2, home=False)
df_away_lag_3 = get_lag_by_team_date(treated_df, 3, home=False)

df_away_lag = df_away_lag_1.merge(df_away_lag_2,
                                  on=['team_away', 'schedule_date'],
                                  how='left')
df_away_lag = df_away_lag.merge(df_away_lag_3,
                                  on=['team_away', 'schedule_date'],
                                  how='left')

In [9]:
featured_df = featured_df.merge(df_home_lag,
                                on=['team_home', 'schedule_date'],
                                how='left')
featured_df = featured_df.merge(df_away_lag,
                                on=['team_away', 'schedule_date'],
                                how='left')
featured_df.tail()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_scrm_plys_home_,defense_fumbles_ff_home_,defense_fumbles_fr_home_,defense_interceptions_int_yds_home_,defense_interceptions_lng_home_,defense_passing_cmp__perc_home_,defense_passing_rate_home_,defense_receiving_yds/rec_home_,defense_receiving_pdef_home_,defense_rushing_ypc_home_,defense_rushing_td_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_scoring_sfty_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_scrm_plys_home_,offense_receiving_yds/rec_home_,offense_receiving_lng_home_,offense_receiving_rec_fum_home_,offense_rushing_ypc_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_fum_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_lng_home_,special_teams_kickoff_returns_fc_home_,special_teams_kickoff_returns_fum_home_,special_teams_kickoff_returns_fg_blk_home_,special_teams_kickoff_returns_xp_blk_home_,special_teams_kickoffs_ret_home_,special_teams_kickoffs_ret_avg_home_,special_teams_kickoffs_osk_home_,special_teams_kickoffs_osk_rec_home_,special_teams_kickoffs_oob_home_,special_teams_kickoffs_td_home_,special_teams_punt_returns_avg_home_,special_teams_punt_returns_ret_home_,special_teams_punt_returns_pret_t_home_,special_teams_punt_returns_40+_home_,special_teams_punt_returns_lng_home_,special_teams_punt_returns_fc_home_,special_teams_punt_returns_fum_home_,special_teams_punt_returns_p_blk_home_,defense_downs_3rd_att_away_,defense_downs_3rd_md_away_,defense_downs_scrm_plys_away_,defense_fumbles_ff_away_,defense_fumbles_fr_away_,defense_interceptions_int_yds_away_,defense_interceptions_lng_away_,defense_passing_cmp__perc_away_,defense_passing_rate_away_,defense_receiving_yds/rec_away_,defense_receiving_pdef_away_,defense_rushing_ypc_away_,defense_rushing_td_away_,defense_rushing_20+_away_,defense_rushing_40+_away_,defense_rushing_lng_away_,defense_scoring_sfty_away_,defense_tackles_solo_away_,offense_downs_3rd_att_away_,offense_downs_3rd_md_away_,offense_downs_scrm_plys_away_,offense_receiving_yds/rec_away_,offense_receiving_lng_away_,offense_receiving_rec_fum_away_,offense_rushing_ypc_away_,offense_rushing_20+_away_,offense_rushing_40+_away_,offense_rushing_lng_away_,offense_rushing_rush_fum_away_,offense_scoring_2-pt_away_,special_teams_field_goals_fg__perc_away_,special_teams_field_goals_1-19_>_percent_a_m_away_,special_teams_field_goals_20-29_>_percent_a_m_away_,special_teams_field_goals_30-39_>_percent_a_m_away_,special_teams_field_goals_40-49_>_percent_a_m_away_,special_teams_field_goals_50-59_>_percent_a_m_away_,special_teams_field_goals_60+_>_percent_a_m_away_,special_teams_field_goals_lng_away_,special_teams_field_goals_fg_blk_away_,special_teams_kickoff_returns_avg_away_,special_teams_kickoff_returns_kret_td_away_,special_teams_kickoff_returns_40+_away_,special_teams_kickoff_returns_lng_away_,special_teams_kickoff_returns_fc_away_,special_teams_kickoff_returns_fum_away_,special_teams_kickoff_returns_fg_blk_away_,special_teams_kickoff_returns_xp_blk_away_,special_teams_kickoffs_ret_away_,special_teams_kickoffs_ret_avg_away_,special_teams_kickoffs_osk_away_,special_teams_kickoffs_osk_rec_away_,special_teams_kickoffs_oob_away_,specia

### Wins and loses of those teams historically

In [10]:
historic = {}

def calculate_historic(row):
    team_1, team_2 = sorted([row['team_home'], row['team_away']])
    key = (team_1, team_2)
    if key not in historic:
        historic[key] = {'victories_team_1': 0, 'victories_team_2': 0, 'matches': 0}
    historic[key]['matches'] += 1

    if row['home_winner']:
      if row['team_home'] == team_1:
        historic[key]['victories_team_1'] += 1
      else:
        historic[key]['victories_team_2'] += 1
    else:
      if row['team_home'] == team_1:
        historic[key]['victories_team_2'] += 1
      else:
        historic[key]['victories_team_1'] += 1

    if row['team_home'] == team_1:
       return historic[key]['victories_team_1'] / historic[key]['matches']
    else:
      return historic[key]['victories_team_2'] / historic[key]['matches']

featured_df['home_win_historic_confront'] = featured_df.apply(calculate_historic, axis=1)
featured_df['away_win_historic_confront'] = 1 - featured_df['home_win_historic_confront']

In [11]:
featured_df.tail()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_scrm_plys_home_,defense_fumbles_ff_home_,defense_fumbles_fr_home_,defense_interceptions_int_yds_home_,defense_interceptions_lng_home_,defense_passing_cmp__perc_home_,defense_passing_rate_home_,defense_receiving_yds/rec_home_,defense_receiving_pdef_home_,defense_rushing_ypc_home_,defense_rushing_td_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_scoring_sfty_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_scrm_plys_home_,offense_receiving_yds/rec_home_,offense_receiving_lng_home_,offense_receiving_rec_fum_home_,offense_rushing_ypc_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_fum_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_lng_home_,special_teams_kickoff_returns_fc_home_,special_teams_kickoff_returns_fum_home_,special_teams_kickoff_returns_fg_blk_home_,special_teams_kickoff_returns_xp_blk_home_,special_teams_kickoffs_ret_home_,special_teams_kickoffs_ret_avg_home_,special_teams_kickoffs_osk_home_,special_teams_kickoffs_osk_rec_home_,special_teams_kickoffs_oob_home_,special_teams_kickoffs_td_home_,special_teams_punt_returns_avg_home_,special_teams_punt_returns_ret_home_,special_teams_punt_returns_pret_t_home_,special_teams_punt_returns_40+_home_,special_teams_punt_returns_lng_home_,special_teams_punt_returns_fc_home_,special_teams_punt_returns_fum_home_,special_teams_punt_returns_p_blk_home_,defense_downs_3rd_att_away_,defense_downs_3rd_md_away_,defense_downs_scrm_plys_away_,defense_fumbles_ff_away_,defense_fumbles_fr_away_,defense_interceptions_int_yds_away_,defense_interceptions_lng_away_,defense_passing_cmp__perc_away_,defense_passing_rate_away_,defense_receiving_yds/rec_away_,defense_receiving_pdef_away_,defense_rushing_ypc_away_,defense_rushing_td_away_,defense_rushing_20+_away_,defense_rushing_40+_away_,defense_rushing_lng_away_,defense_scoring_sfty_away_,defense_tackles_solo_away_,offense_downs_3rd_att_away_,offense_downs_3rd_md_away_,offense_downs_scrm_plys_away_,offense_receiving_yds/rec_away_,offense_receiving_lng_away_,offense_receiving_rec_fum_away_,offense_rushing_ypc_away_,offense_rushing_20+_away_,offense_rushing_40+_away_,offense_rushing_lng_away_,offense_rushing_rush_fum_away_,offense_scoring_2-pt_away_,special_teams_field_goals_fg__perc_away_,special_teams_field_goals_1-19_>_percent_a_m_away_,special_teams_field_goals_20-29_>_percent_a_m_away_,special_teams_field_goals_30-39_>_percent_a_m_away_,special_teams_field_goals_40-49_>_percent_a_m_away_,special_teams_field_goals_50-59_>_percent_a_m_away_,special_teams_field_goals_60+_>_percent_a_m_away_,special_teams_field_goals_lng_away_,special_teams_field_goals_fg_blk_away_,special_teams_kickoff_returns_avg_away_,special_teams_kickoff_returns_kret_td_away_,special_teams_kickoff_returns_40+_away_,special_teams_kickoff_returns_lng_away_,special_teams_kickoff_returns_fc_away_,special_teams_kickoff_returns_fum_away_,special_teams_kickoff_returns_fg_blk_away_,special_teams_kickoff_returns_xp_blk_away_,special_teams_kickoffs_ret_away_,special_teams_kickoffs_ret_avg_away_,special_teams_kickoffs_osk_away_,special_teams_kickoffs_osk_rec_away_,special_teams_kickoffs_oob_away_,specia

## Average result feature

Average points last 3 games

Average points per game last season

Total points last season

AVerage in-game stats

## Data Leakage avoidance

We might be aware of **data leakage**.

Examples of data leakage in machine learning models:
* Target leakage: When a model learns from data that contains information about the target variable
* Data preprocessing leakage: When information from the test set is used during data preprocessing
* Using the same dataset for training and testing: This is an example of overfitting, which can lead to overoptimistic performance estimates
* Using future data to make predictions about the past: This can happen in time series problems

### Removing score information